# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction


RNA is a fragile molecule that is highly sensitive to degradation.
Several factors seems to affect mRNA stability, for exemple:
- length
- composition (GC content)
- secondary structure

The goal of this project is to determine the stability of mRNA molecules by predicting the parameters for each position in the molecule: 
* Reactivity 
* Deg_Mg_pH10 
* Deg_Mg_50C 

The train data is composed of 2400 mRNA molecules that have been tested for stability to ph and heat at each of the 68 out of 107 positions.

1. Overlook at the raw data

In [1]:
import pandas as pd
import json
import numpy as np
import seaborn as sns


In [2]:
df_train= pd.read_json(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\train.json', lines=True)
# df_test = pd.read_json(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\test.json', lines=True)


In [6]:
print(df_train['sequence'][0])
print(df_train['sequence'][1])
print(df_train['sequence'][2])
print(df_train['sequence'][3])
print(df_train['sequence'][4])
print(df_train['sequence'][5])

GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC
GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACCGCGCGCGCUUAUGCAAGUUGCCCGCGGCGUUCGCGCUGUGAAAAGAAACAACAACAACAAC
GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAUCGAAAUAGCAUCGAUGUGUAUAUGGGUGGUUCGCCGCUCAAAAAGAAACAACAACAACAAC
GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGCGCGCGCGCCCUAGUCGUGGUGCUCGAGGUUUCGACCUCGAAAAAGAAACAACAACAACAAC
GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUAUAUUAUAUAAAUAUAUAUAUAUAAAAUAUUUCGAUAUUUUAAAAGAAACAACAACAACAAC
GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCACAGCGCCGCAGCGGCACACACACCGUAAGUUCGCUUGCGGAAAAGAAACAACAACAACAAC


In [3]:
# df_train = df_test

In [50]:
# df_train1['flag'] = list(['train'] * len(df_train1))
# df_train1.shape

(2400, 20)

In [51]:
# df_test['flag'] = list(['test'] * len(df_test))
# df_test.shape

(3634, 8)

In [59]:
# df_train= pd.concat([df_test, df_train1], ignore_index=True)
# df_train.shape

(6034, 20)

In [4]:
test_df = df_train.loc[:2]
# print(type(test_df["sequence"][0]))
a = test_df["sequence"][0]      
# to_be_counted is <class 'str'> 

test_df

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."


Each molecule of RNA is to be atributed to the following parameters:
- GC content
- Single pair probability
- APP (average paired probability)
- MFE (Minimum Free Energy)

Each position in the molecule needs to be analyzed and atributed to the following parameters:
- Neighboor nucleotide: NN-X-NN, 35 possibilities
- Secondary structure: paired (0), not paired (1)
- pairing type: GC (0), AU (1), GU (2)
- Loop type: E, S, H, X, B, I
- Distance from a structured region (next paired nucleotide)
- Degradation values for each conditions
- Degradation values QC for each conditions

The skeleton of such dataframe needs to be drown.

**1. Manipulation on the whole molecule of mRNA**

__a. GC content__

The GC content is an important factor in the stability of RNA as the GC nucleotides are slightly heavier than their counteparts AU. Moreover, G and C tends to form stronger bonds thereby strengthnig the secondary structure and enhancing stability.

$$GC(content) = \frac{GC(count)}{nucleotides(count)}$$





In [60]:
df_train['sequence'][0]

'GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAAUGGUCAAAGAGGUCGAAAUACAGAUGACCUUCGGGUUAUCAAAAGAAACAACAACAACAAC'

In [4]:
"""
A function that counts the number of GC and total nucleotides. The function returns the GC content.
"""



def gc_content(to_be_counted):
    GC_count = to_be_counted.count("G") + to_be_counted.count("C")
    nucleotide_count = len(to_be_counted)
    GC_content = GC_count/ nucleotide_count
    return GC_content

def gc_content_column(df):
    GC_content = []
    for row in range(len(df)):
        sequence = df["sequence"][row]
        GC = gc_content(sequence)
        GC_content.append(GC)
    df.insert(2, "GC content", GC_content)
#     print(test_df.shape)
    return df
#     print(GC_content)
        

gc_content_column(df_train)
# gc_content_column(df_train)
# print(test_df)

,index,id,GC content,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,0.392523,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,0.423077,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,0.495327,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,0.514019,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,0.623077,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91
...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91
3630,3630,id_ff9bf3581,0.400000,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91
3631,3631,id_ffc8f96a8,0.453846,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91
3632,3632,id_ffd7e8cc1,0.476923,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91


__b. AUP (average unpaired probability)__

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                3634 non-null   int64  
 1   id                   3634 non-null   object 
 2   GC content           3634 non-null   float64
 3   sequence             3634 non-null   object 
 4   structure            3634 non-null   object 
 5   predicted_loop_type  3634 non-null   object 
 6   seq_length           3634 non-null   int64  
 7   seq_scored           3634 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 227.2+ KB


__c. MFE (Minimum Free Energy)__

To calculate the MFE, the vienna Rna package was used https://www.tbi.univie.ac.at/RNA/.



In [7]:
seq2396 = df_train.iloc[2396,4]
structure2396 = df_train.iloc[2396,5]
print(seq2396)
structure2396

seq1 = 'GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAAUGGUCAAAGAGGUCGAAAUACAGAUGACCUUCGGGUUAUCAAAAGAAACAACAACAACAAC'
structure1 = '......((((((((((.(((((.....))))))))((((((((...)))))...)))))))...))).(((((((....))))))).....................'


....................................................................(((((((....))))))).....................


In [5]:
from subprocess import Popen, PIPE
"""
get_mfe is a function that used RNA package installed on my computer. 
It returns the minimal folding energy based on the given model structure and the sequence of the RNA molecule.
"""

def get_mfe(seq, pred_structure):
    path = r"C:\Program Files (x86)\ViennaRNA Package\RNA2Dfold.exe"
    cmd_input = '''{}
{}
{}'''.format(seq, pred_structure, pred_structure).encode()
    start_cmd = Popen(args=path, stdin=PIPE, stdout=PIPE)
    response = start_cmd.communicate(input=cmd_input)
    decoded_response = response[0].decode()
    all_mfe = decoded_response.split()
#     print(all_mfe)
#     print(type(all_mfe[2].strip("()")))
    if all_mfe[2].strip("()"):
        mfe = all_mfe[2].strip("()")
    else:
        mfe = all_mfe[3].strip("()")
#     print(mfe)
    return mfe

# get_mfe(seq2396, structure2396)

In [6]:
seq1 = 'GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAAUGGUCAAAGAGGUCGAAAUACAGAUGACCUUCGGGUUAUCAAAAGAAACAACAACAACAAC'
structure1 = '......((((((((((.(((((.....))))))))((((((((...)))))...)))))))...))).(((((((....))))))).....................'

# get_mfe(seq1, structure1)
"""
list_mfe takes several sequences and structures and returns a list of mfe accordingly.
"""

def list_mfe(df):
    MFE_value = []
    for index in range (len(df)):
        sequence, structure = df["sequence"][index],df["structure"][index]
        mfe = get_mfe(sequence, structure)
#         print(mfe)
        MFE_value.append(mfe)
#         index+=1
    df.insert(3, "MFE value", MFE_value)
#     df["MFE value"] = MFE_value
#     print(df.shape)
    return df
# print(test_df)    
#     df["MFE value"] = MFE_value


In [7]:
df_train_copy1 = df_train.copy()
list_mfe(df_train_copy1)

,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,0.392523,-21.70,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,0.423077,-37.00,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,0.495327,-28.70,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,0.514019,-47.50,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,0.623077,-92.70,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91
...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.70,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91
3630,3630,id_ff9bf3581,0.400000,-32.20,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91
3631,3631,id_ffc8f96a8,0.453846,-64.70,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91
3632,3632,id_ffd7e8cc1,0.476923,-26.00,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91


In [8]:
"""
converting mfe values as object to float
"""

df_train_copy1['MFE value'] = df_train_copy1['MFE value'].astype('float')


In [9]:
df_train_copy1.to_pickle(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\pickle\test_gc_mfe.pkl')

SyntaxError: unexpected EOF while parsing (<ipython-input-9-1fa46e26a6bb>, line 1)

In [2]:
df_train_copy1 = pd.read_pickle(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\pickle\test_gc_mfe.pkl')

In [3]:
df_train_copy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                3634 non-null   int64  
 1   id                   3634 non-null   object 
 2   GC content           3634 non-null   float64
 3   MFE value            3634 non-null   float64
 4   sequence             3634 non-null   object 
 5   structure            3634 non-null   object 
 6   predicted_loop_type  3634 non-null   object 
 7   seq_length           3634 non-null   int64  
 8   seq_scored           3634 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 255.6+ KB


In [52]:
 seq1 = 'GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAAUGGUCAAAGAGGUCGAAAUACAGAUGACCUUCGGGUUAUCAAAAGAAACAACAACAACAAC'

Once again, using viennaRNA package to calculate the pair probability :rnaplfold.exe

In [6]:

def get_pair_prob(seq):
    from subprocess import Popen, PIPE
    path = r"C:\Program Files (x86)\ViennaRNA Package\RNAplfold.exe"
    cmd_input = seq.encode()
    start_cmd = Popen(args=path, stdin=PIPE, stdout=PIPE)
    response = start_cmd.communicate(input=cmd_input)
    return len(seq)
#     print('func1: ok')


def regx_table():
    import re
    # a = pd.read_table(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_dp.ps')
    with open (r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_data.csv', 'w') as myfile2:
        with open (r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_dp.ps', 'r+') as myfile:  # Open lorem.txt for reading
            for myline in myfile:              # For each line, read to a string,
                if 'ubox' in myline:
                    new = re.sub('[ubox/{%ijsqrt(p(ij)),]', "", myline)
                    myfile2.write(new)
    df = pd.read_table(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_data.csv',header=None, sep='\s', engine='python')
    df.columns=['nucl i', 'nucl j', 'aup']
#     print('func2: ok')
    return df

def mean_pair(df):
    a = df.drop(columns = ['nucl j'])
    a.columns = ['nucleotide', 'pair prob']
    b = df.drop(columns = ['nucl i'])
    b.columns = ['nucleotide', 'pair prob']
    c = pd.concat([a, b], ignore_index=True)

    df_paired_nucleotide = c.groupby('nucleotide').mean().reset_index()
    list_paired_nucleotide = list(df_paired_nucleotide['nucleotide'])
#     print('func3: ok')
    return  list_paired_nucleotide, df_paired_nucleotide


def diff(list_paired_nucleotide, seq_length):
    total = list(range(1,seq_length+1))
    list_unpaired = (list(list(set(total)-set(list_paired_nucleotide)) + list(set(list_paired_nucleotide)-set(total))))
#     print('func4: ok')
    return list_unpaired

def unpaired_in_frame(list_unpaired, df_paired_nucleotide):
    my_dict = {}
    if list_unpaired:
        for number in list_unpaired:
            my_dict.update({number:0})
        new_df = pd.DataFrame.from_dict(my_dict, orient = 'index')
        new_df.reset_index(inplace=True)
#         print(new_df)
        new_df.columns = ['nucleotide', 'pair prob']
    else:
        new_df = pd.DataFrame(columns =['nucleotide', 'pair prob'])
    all_pos = pd.concat([new_df, df_paired_nucleotide], ignore_index=True)
    all_pos.sort_values(by = 'nucleotide', axis=0, inplace=True)
    list_all_prob = list(all_pos['pair prob'])
    #     print('func5: ok')
    return all_pos, list_all_prob


def get_pair_probability(seq):
    seq_length = get_pair_prob(seq)
    df= regx_table()
    list_paired_nucleotide, df_paired_nucleotide = mean_pair(df)
    list_unpaired = diff(list_paired_nucleotide, seq_length)
    all_pos, list_all_prob = unpaired_in_frame(list_unpaired, df_paired_nucleotide)
    import os
#     os.remove(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_data.csv')
#     os.remove(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\plfold_dp.ps')
#     print('func6: ok')
    return list_all_prob
    
def get_rnaplfold(df):
    n= 0 
    rnaplfold = []
    for index in range (len(df)):
        print(index)
#         sequence = df["sequence"][index]
        paired_prob = get_pair_probability(df["sequence"][index])
        rnaplfold.append(paired_prob)
        n+=1
#         print('the function succed the {} iteration'.format(n))
    df['rnaplfold'] = rnaplfold
    
    return df

In [7]:
# df_train_copy1.drop(columns = ['rnaplfold', 'Average pair prob'], inplace=True)

get_rnaplfold(df_train_copy1)

# get_pair_prob(seq1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633


,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored,rnaplfold
0,0,id_00073f8be,0.392523,-21.7,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.4052, 0.48195, 0.4..."
1,1,id_000ae4237,0.423077,-37.0,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[0.9578, 0.6841, 0.5552, 0.4393, 0.0, 0.8542, ..."
2,2,id_00131c573,0.495327,-28.7,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68,"[0.2818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2818, 0.0,..."
3,3,id_00181fd34,0.514019,-47.5,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68,"[0.27585000000000004, 0.3035, 0.2606, 0.2744, ..."
4,4,id_0020473f7,0.623077,-92.7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[0.39890000000000003, 0.3506, 0.0, 0.0, 0.3857..."
...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.7,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[0.23125, 0.2132, 0.0, 0.35715, 0.34515, 0.212..."
3630,3630,id_ff9bf3581,0.400000,-32.2,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[0.0, 0.3323, 0.1923, 0.0, 0.3889, 0.3611, 0.0..."
3631,3631,id_ffc8f96a8,0.453846,-64.7,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91,"[0.25605, 0.2511666666666667, 0.19966666666666..."
3632,3632,id_ffd7e8cc1,0.476923,-26.0,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91,"[0.24974999999999997, 0.24850000000000003, 0.1..."


The average pair probability is obtained by calculating the mean of single pair probability in the RNA molecule

In [8]:
from statistics import mean 
list_app = []
for index in range (len(df_train_copy1)):
    app = mean(df_train_copy1["rnaplfold"][index])
    list_app.append(app)
    
df_train_copy1['Average pair prob'] = list_app


In [142]:
test_df["sequence"][0]

'GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC'

In [9]:
df_train_copy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                3634 non-null   int64  
 1   id                   3634 non-null   object 
 2   GC content           3634 non-null   float64
 3   MFE value            3634 non-null   float64
 4   sequence             3634 non-null   object 
 5   structure            3634 non-null   object 
 6   predicted_loop_type  3634 non-null   object 
 7   seq_length           3634 non-null   int64  
 8   seq_scored           3634 non-null   int64  
 9   rnaplfold            3634 non-null   object 
 10  Average pair prob    3634 non-null   float64
dtypes: float64(3), int64(3), object(5)
memory usage: 312.4+ KB


In [227]:
df_train_copy1.describe()

,index,GC content,MFE value,signal_to_noise,SN_filter,seq_length,seq_scored
count,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.0,2400.0
mean,1199.500000,0.444147,-29.650308,4.530456,0.662083,107.0,68.0
std,692.964646,0.117803,16.968926,2.835142,0.473099,0.0,0.0
min,0.000000,0.093458,-103.600000,-0.103000,0.000000,107.0,68.0
25%,599.750000,0.383178,-38.525000,2.391000,0.000000,107.0,68.0
50%,1199.500000,0.448598,-27.250000,4.442500,1.000000,107.0,68.0
75%,1799.250000,0.514019,-17.400000,6.294250,1.000000,107.0,68.0
max,2399.000000,0.813084,-0.800000,17.194000,1.000000,107.0,68.0


In [18]:
df_train_copy1.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_pairprob_GC_beforeExplode2.csv')

Next, interaction type will be solved in order to find in which kind of interaction a paired nucleotide is in: 
pairing type: GC-CG , AU-UA , GU-UG , none (0)
find paired nucleotides in list

In [10]:
"""
The pair function find which nucleotides are paired based on the given structure model. 
It works on a list of nucleotide and strcuture dots.
it return GC-CG , AU-UA , GU-UG , none (0)
"""
count = 0
pairing_dict = {}
paired = []
def pair(structure,seq, dataframe):
    list_of_dots = list(dataframe[structure])
#     print(list_of_dots)
    sequence = list(dataframe[seq])
#     print(seq)
    list_nucleotide_paring = []
    for seq_index in range(len(dataframe)):
#         print(list_of_dots[seq_index])
#         print(sequence[seq_index])
        seq_length = len(sequence[seq_index])
        nucleotide_pairing = []
        for index in range (seq_length):
#             print(list_of_dots[seq_index][index])
            if list_of_dots[seq_index][index] == ".":
#                 print('dots index: {}'.format(index))
                pairing_dict.update({index:0})
            if list_of_dots[seq_index][index] == "(":
#                 print('open index: {}'.format(index))
                paired_nucleotide = []
                paired_nucleotide.append(index)
                paired.append(paired_nucleotide)
    #            print(paired)
            if list_of_dots[seq_index][index] == ")":
#                 print('close index: {}'.format(index))
                paired[-1].append(index)
#                 print(paired)
                pairing_dict.update({paired[-1][0]:(sequence[seq_index][paired[-1][0]]+ sequence[seq_index][paired[-1][1]])})
                pairing_dict.update({paired[-1][1]:(sequence[seq_index][paired[-1][0]]+ sequence[seq_index][paired[-1][1]])})                 
                paired.pop()
    #     print("paired is {}".format(paired))
        sorted_pairing_dict = sorted(pairing_dict.items())
#         print(len(sorted_pairing_dict)) 
        for tuple in sorted_pairing_dict:
            nucleotide_pairing.append(tuple[1])
        list_nucleotide_paring.append(nucleotide_pairing)
    
    dataframe["nucleotide pairing"] = list_nucleotide_paring
    return dataframe

In [12]:
df_train_copy2 = df_train_copy1.copy() 
pair('structure', 'sequence', df_train_copy2 )

,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored,rnaplfold,Average pair prob,nucleotide pairing
0,0,id_00073f8be,0.392523,-21.7,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.4052, 0.48195, 0.4...",0.327010,"[0, 0, 0, 0, 0, 0, GC, UA, AU, CG, GC, AU, CG,..."
1,1,id_000ae4237,0.423077,-37.0,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[0.9578, 0.6841, 0.5552, 0.4393, 0.0, 0.8542, ...",0.449699,"[0, 0, 0, 0, 0, CG, GC, GC, GC, 0, 0, CG, CG, ..."
2,2,id_00131c573,0.495327,-28.7,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68,"[0.2818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2818, 0.0,...",0.271359,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, CG, GC, 0, C..."
3,3,id_00181fd34,0.514019,-47.5,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68,"[0.27585000000000004, 0.3035, 0.2606, 0.2744, ...",0.424273,"[0, 0, 0, 0, 0, 0, GC, AU, UA, CG, UA, CG, UA,..."
4,4,id_0020473f7,0.623077,-92.7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[0.39890000000000003, 0.3506, 0.0, 0.0, 0.3857...",0.453588,"[0, 0, 0, 0, 0, CG, CG, CG, GU, CG, CG, CG, GU..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.7,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[0.23125, 0.2132, 0.0, 0.35715, 0.34515, 0.212...",0.370164,"[0, 0, 0, 0, 0, 0, 0, 0, GC, CG, CG, AU, UG, G..."
3630,3630,id_ff9bf3581,0.400000,-32.2,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[0.0, 0.3323, 0.1923, 0.0, 0.3889, 0.3611, 0.0...",0.311562,"[0, 0, 0, 0, 0, 0, 0, GC, CG, GC, CG, CG, 0, 0..."
3631,3631,id_ffc8f96a8,0.453846,-64.7,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91,"[0.25605, 0.2511666666666667, 0.19966666666666...",0.494107,"[0, 0, 0, 0, 0, GC, AU, UA, GC, UA, UA, CG, 0,..."
3632,3632,id_ffd7e8cc1,0.476923,-26.0,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91,"[0.24974999999999997, 0.24850000000000003, 0.1...",0.309302,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


__c. Neighboor nucleotide: NN-X-NN/ N-X-N__
    
The function *find neighboor* takes as argument:

* the dataframe
* the number of neighboor to select (1 or 2 on each side)
* the name of the column (="sequence")

it returns the dataframe with an added column that exhibits the framing nucleotides.  



In [13]:
def find_neighboor(dataframe, number_of_neighboor = 2):
    sequence = list(dataframe["sequence"])
    seq_len = list(dataframe["seq_length"])
    list_nucleotide_Minus2=[]
    list_nucleotide_Minus1=[]
    list_nucleotide_Plus1 = []
    list_nucleotide_Plus2 = []
    if number_of_neighboor == 2:
        for seq_index in range (len(dataframe)):
            nucleotide_Minus2=[]
            nucleotide_Minus1=[]
            nucleotide_Plus1 = []
            nucleotide_Plus2 = []
            seq_length = seq_len[seq_index]
            for index in range (seq_length):
                if index-2 >=0 :
                    nucleotide_Minus2.append(sequence[seq_index][index-2])
                else: 
                    nucleotide_Minus2.append("start")
                if index-1 >=0 :
                    nucleotide_Minus1.append(sequence[seq_index][index-1])
                else: 
                    nucleotide_Minus1.append("start") 
                if index + 1 < seq_length :
                    nucleotide_Plus1.append(sequence[seq_index][index+1])
                else: 
                    nucleotide_Plus1.append("end") 
                if index + 2 < seq_length :
                    nucleotide_Plus2.append(sequence[seq_index][index+2])
                else: 
                    nucleotide_Plus2.append("end")
#             print(nucleotide_Minus1)
#             print(nucleotide_Minus2)
#             print(nucleotide_Plus1)
#             print(nucleotide_Plus2)
            list_nucleotide_Minus2.append(nucleotide_Minus2)
            list_nucleotide_Minus1.append(nucleotide_Minus1)
            list_nucleotide_Plus1.append(nucleotide_Plus1)
            list_nucleotide_Plus2.append(nucleotide_Plus2) 
#         print(serie_4_neighboor)
    dataframe["nucleotide_Minus2"] = list_nucleotide_Minus2
    dataframe["nucleotide_Minus1"] = list_nucleotide_Minus1
    dataframe["nucleotide_Plus1"] = list_nucleotide_Plus1
    dataframe["nucleotide_Plus2"] = list_nucleotide_Plus2
    return dataframe
#     if number_of_neighboor == 1:
#         for index in range (len(dataframe)):
#             neighboor = []
#             try:
#                     neighboor.append(seq_serie[index-1])
#             except: 
#                 neighboor.append("none") 
#             try:
#                 neighboor.append(seq_serie[index+1])
#             except: 
#                 neighboor.append("none") 
#             finally:
#                 serie_2_neighboor.append(neighboor)
# #         print(serie_2_neighboor)
#         dataframe["serie_2_neighboor"] = serie_2_neighboor
    
#     if number_of_neighboor == 1:
#         neighboor5p = []
#         neighboor3p = []
#         for index in range (len(dataframe)):
            
#             try:
#                 neighboor5p.append(seq_serie[index-1])
#             except: 
#                 neighboor5p.append("start") 
#             try:
#                 neighboor3p.append(seq_serie[index+1])
#             except: 
#                 neighboor3p.append("end") 
# #             finally:
# #                 serie_2_neighboor.append(neighboor)
# #         print(serie_2_neighboor)
#         print(len(neighboor5p))
#         dataframe["neighboor 5 prime"] = neighboor5p
#         dataframe["neighboor 3 prime"] = neighboor3p
#         return dataframe

    



In [14]:
df_train_copy3 = df_train_copy2.copy() 
df_train_copy3 = find_neighboor(dataframe=df_train_copy3)

In [175]:
df_train_copy3

,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored,rnaplfold,Average pair prob,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2
0,0,id_00073f8be,0.392523,-21.7,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.4052, 0.48195, 0.4...",0.327010,"[0, 0, 0, 0, 0, 0, GC, UA, AU, CG, GC, AU, CG,...","[start, start, G, G, A, A, A, A, G, U, A, C, G...","[start, G, G, A, A, A, A, G, U, A, C, G, A, C,...","[G, A, A, A, A, G, U, A, C, G, A, C, U, U, G, ...","[A, A, A, A, G, U, A, C, G, A, C, U, U, G, A, ..."
1,1,id_000ae4237,0.423077,-37.0,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[0.9578, 0.6841, 0.5552, 0.4393, 0.0, 0.8542, ...",0.499666,"[0, 0, 0, 0, 0, CG, GC, GC, GC, 0, 0, CG, CG, ...","[start, start, G, G, A, A, A, C, G, G, G, U, U...","[start, G, G, A, A, A, C, G, G, G, U, U, C, C,...","[G, A, A, A, C, G, G, G, U, U, C, C, G, C, G, ...","[A, A, A, C, G, G, G, U, U, C, C, G, C, G, G, ..."
2,2,id_00131c573,0.495327,-28.7,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68,"[0.2818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2818, 0.0,...",0.271359,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, CG, GC, 0, C...","[start, start, G, G, A, A, A, A, C, A, A, A, A...","[start, G, G, A, A, A, A, C, A, A, A, A, C, G,...","[G, A, A, A, A, C, A, A, A, A, C, G, G, C, C, ...","[A, A, A, A, C, A, A, A, A, C, G, G, C, C, U, ..."
3,3,id_00181fd34,0.514019,-47.5,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68,"[0.27585000000000004, 0.3035, 0.2606, 0.2744, ...",0.424273,"[0, 0, 0, 0, 0, 0, GC, AU, UA, CG, UA, CG, UA,...","[start, start, G, G, A, A, A, G, G, A, U, C, U...","[start, G, G, A, A, A, G, G, A, U, C, U, C, U,...","[G, A, A, A, G, G, A, U, C, U, C, U, A, U, C, ...","[A, A, A, G, G, A, U, C, U, C, U, A, U, C, G, ..."
4,4,id_0020473f7,0.623077,-92.7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[0.39890000000000003, 0.3506, 0.0, 0.0, 0.3857...",0.412353,"[0, 0, 0, 0, 0, CG, CG, CG, GU, CG, CG, CG, GU...","[start, start, G, G, A, A, A, C, C, C, G, C, C...","[start, G, G, A, A, A, C, C, C, G, C, C, C, G,...","[G, A, A, A, C, C, C, G, C, C, C, G, C, G, C, ...","[A, A, A, C, C, C, G, C, C, C, G, C, G, C, C, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.7,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[0.23125, 0.2132, 0.0, 0.35715, 0.34515, 0.212...",0.331871,"[0, 0, 0, 0, 0, 0, 0, 0, GC, CG, CG, AU, UG, G...","[start, start, G, G, A, A, A, C, U, A, G, C, C...","[start, G, G, A, A, A, C, U, A, G, C, C, A, U,...","[G, A, A, A, C, U, A, G, C, C, A, U, G, G, G, ...","[A, A, A, C, U, A, G, C, C, A, U, G, G, G, C, ..."
3630,3630,id_ff9bf3581,0.400000,-32.2,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[0.0, 0.3323, 0.1923, 0.0, 0.3889, 0.3611, 0.0...",0.352201,"[0, 0, 0, 0, 0, 0, 0, GC, CG, GC, CG, CG, 0, 0...","[start, start, G, G, A, A, A, U, A, G, C, G, C...","[start, G, G, A, A, A, U, A, G, C, G, C, C, A,...","[G, A, A, A, U, A, G, C, G, C, C, A, U, A, G, ...","[A, A, A, U, A, G, C, G, C, C, A, U, A, G, C, ..."
3631,3631,id_ffc8f96a8,0.453846,-64.7,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEE

In [188]:
df_train_copy3.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_PairProb_GC_neighboors_pairing_beforeExplode.csv')

In [15]:
df_train_copy3.to_pickle(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\pickle\test_gc_mfe_pairprob_pair_neighbor.pkl')

In [189]:
df_train_copy3.columns

Index(['index', 'id', 'GC content', 'MFE value', 'sequence', 'structure',
       'predicted_loop_type', 'signal_to_noise', 'SN_filter', 'seq_length',
       'seq_scored', 'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'rnaplfold', 'nucleotide pairing',
       'nucleotide_Minus2', 'nucleotide_Minus1', 'nucleotide_Plus1',
       'nucleotide_Plus2'],
      dtype='object')

__Closest paired nucleotide__

A function that map how far from a paired structure an unpaired nucleotide is.

ex: ...() --> 3 2 1 0 0

closest_paired_structure = []

1. if ( or ), append 0
2. if ., append 1


In [16]:
# df = pd.read_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_GC_neighboors_pairing_beforeExplode.csv')

seq = '.....((((((.......)))).)).((.....((..((((((....))))))..)).....))....(((((((....))))))).....................'
def closest_paired_structure_by_mor(seq):
    closest_forwards = []
    closest_backwards = []
    closest = []
    
    def count(seq):
        started_counting = False
        closest = []
        for i in range(len(seq)):
            if (seq[i] != '.'):
                started_counting = True
                count = 0
                closest.append(0)
            elif started_counting:
                count += 1
                closest.append(count)
            else:
                closest.append(None)
        return closest

    def seq_min(seq1, seq2):
        seq = []
        for i in range(len(seq1)):
            if seq1[i] == None:
                seq.append(seq2[i])
            elif seq2[i] == None:
                seq.append(seq1[i])
            elif seq1[i] > seq2[i]:
                seq.append(seq2[i])
            else:
                seq.append(seq1[i])
        return seq

    closest_forwards = count(seq)
    closest_backwards = count(seq[::-1])
    closest = seq_min(closest_forwards, closest_backwards[::-1])    
    return closest

closest = closest_paired_structure_by_mor(seq)
print(closest)



            

[5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 3, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 3, 2, 1, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [17]:

list_closest = []
list_of_dots = list(df_train_copy3['structure'])
    
for seq_index in range(len(df_train_copy3)):
    seq = list_of_dots[seq_index]
    closest = closest_paired_structure_by_mor(seq)
    list_closest.append(closest)

df_train_copy3['distance from structured RNA'] = list_closest

df_train_copy3.columns

Index(['index', 'id', 'GC content', 'MFE value', 'sequence', 'structure',
       'predicted_loop_type', 'seq_length', 'seq_scored', 'rnaplfold',
       'Average pair prob', 'nucleotide pairing', 'nucleotide_Minus2',
       'nucleotide_Minus1', 'nucleotide_Plus1', 'nucleotide_Plus2',
       'distance from structured RNA'],
      dtype='object')

In [192]:
df_train_copy3.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_pairpob_GC_neighboors_pairing_closest_beforeExplode.csv')

In [18]:
df_train_copy3.to_pickle(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\pickle\test_gc_mfe_pairprob_pair_neighbor_distance.pkl')

In [19]:
df_train_copy3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         3634 non-null   int64  
 1   id                            3634 non-null   object 
 2   GC content                    3634 non-null   float64
 3   MFE value                     3634 non-null   float64
 4   sequence                      3634 non-null   object 
 5   structure                     3634 non-null   object 
 6   predicted_loop_type           3634 non-null   object 
 7   seq_length                    3634 non-null   int64  
 8   seq_scored                    3634 non-null   int64  
 9   rnaplfold                     3634 non-null   object 
 10  Average pair prob             3634 non-null   float64
 11  nucleotide pairing            3634 non-null   object 
 12  nucleotide_Minus2             3634 non-null   object 
 13  nuc

In [179]:
df_train_copy3

,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored,rnaplfold,Average pair prob,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2,distance from structured RNA
0,0,id_00073f8be,0.392523,-21.7,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.4052, 0.48195, 0.4...",0.327010,"[0, 0, 0, 0, 0, 0, GC, UA, AU, CG, GC, AU, CG,...","[start, start, G, G, A, A, A, A, G, U, A, C, G...","[start, G, G, A, A, A, A, G, U, A, C, G, A, C,...","[G, A, A, A, A, G, U, A, C, G, A, C, U, U, G, ...","[A, A, A, A, G, U, A, C, G, A, C, U, U, G, A, ...","[6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,id_000ae4237,0.423077,-37.0,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[0.9578, 0.6841, 0.5552, 0.4393, 0.0, 0.8542, ...",0.499666,"[0, 0, 0, 0, 0, CG, GC, GC, GC, 0, 0, CG, CG, ...","[start, start, G, G, A, A, A, C, G, G, G, U, U...","[start, G, G, A, A, A, C, G, G, G, U, U, C, C,...","[G, A, A, A, C, G, G, G, U, U, C, C, G, C, G, ...","[A, A, A, C, G, G, G, U, U, C, C, G, C, G, G, ...","[5, 4, 3, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
2,2,id_00131c573,0.495327,-28.7,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68,"[0.2818, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2818, 0.0,...",0.271359,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, CG, GC, 0, C...","[start, start, G, G, A, A, A, A, C, A, A, A, A...","[start, G, G, A, A, A, A, C, A, A, A, A, C, G,...","[G, A, A, A, A, C, A, A, A, A, C, G, G, C, C, ...","[A, A, A, A, C, A, A, A, A, C, G, G, C, C, U, ...","[11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 1, 0..."
3,3,id_00181fd34,0.514019,-47.5,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68,"[0.27585000000000004, 0.3035, 0.2606, 0.2744, ...",0.424273,"[0, 0, 0, 0, 0, 0, GC, AU, UA, CG, UA, CG, UA,...","[start, start, G, G, A, A, A, G, G, A, U, C, U...","[start, G, G, A, A, A, G, G, A, U, C, U, C, U,...","[G, A, A, A, G, G, A, U, C, U, C, U, A, U, C, ...","[A, A, A, G, G, A, U, C, U, C, U, A, U, C, G, ...","[6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,id_0020473f7,0.623077,-92.7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[0.39890000000000003, 0.3506, 0.0, 0.0, 0.3857...",0.412353,"[0, 0, 0, 0, 0, CG, CG, CG, GU, CG, CG, CG, GU...","[start, start, G, G, A, A, A, C, C, C, G, C, C...","[start, G, G, A, A, A, C, C, C, G, C, C, C, G,...","[G, A, A, A, C, C, C, G, C, C, C, G, C, G, C, ...","[A, A, A, C, C, C, G, C, C, C, G, C, G, C, C, ...","[5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.7,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[0.23125, 0.2132, 0.0, 0.35715, 0.34515, 0.212...",0.331871,"[0, 0, 0, 0, 0, 0, 0, 0, GC, CG, CG, AU, UG, G...","[start, start, G, G, A, A, A, C, U, A, G, C, C...","[start, G, G, A, A, A, C, U, A, G, C, C, A, U,...","[G, A, A, A, C, U, A, G, C, C, A, U, G, G, G, ...","[A, A, A, C, U, A, G, C, C, A, U, G, G, G, C, ...","[8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3630,3630,id_ff9bf3581,0.400000,-32.2,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[0.0, 0.3323, 0.1923, 0.0, 0.3889, 0.3611, 0.0...",0.352201,"[0, 0, 0, 0, 0, 0, 0, GC, CG, GC, CG, CG, 0, 0...","[st

In [180]:
df_train_copy3['seq_scored'][1]

91

**2. Manipulation at the nucleotide level**

__a. Drowing the table's skeleton__

In [20]:


df_107 = df_train_copy3.loc[df_train_copy3['seq_length']==107]
df_130 = df_train_copy3.loc[df_train_copy3['seq_length']==130]

df_130

,index,id,GC content,MFE value,sequence,structure,predicted_loop_type,seq_length,seq_scored,rnaplfold,Average pair prob,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2,distance from structured RNA
1,1,id_000ae4237,0.423077,-37.0,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[0.9578, 0.6841, 0.5552, 0.4393, 0.0, 0.8542, ...",0.449699,"[0, 0, 0, 0, 0, CG, GC, GC, GC, 0, 0, CG, CG, ...","[start, start, G, G, A, A, A, C, G, G, G, U, U...","[start, G, G, A, A, A, C, G, G, G, U, U, C, C,...","[G, A, A, A, C, G, G, G, U, U, C, C, G, C, G, ...","[A, A, A, C, G, G, G, U, U, C, C, G, C, G, G, ...","[5, 4, 3, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
4,4,id_0020473f7,0.623077,-92.7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[0.39890000000000003, 0.3506, 0.0, 0.0, 0.3857...",0.453588,"[0, 0, 0, 0, 0, CG, CG, CG, GU, CG, CG, CG, GU...","[start, start, G, G, A, A, A, C, C, C, G, C, C...","[start, G, G, A, A, A, C, C, C, G, C, C, C, G,...","[G, A, A, A, C, C, C, G, C, C, C, G, C, G, C, ...","[A, A, A, C, C, C, G, C, C, C, G, C, G, C, C, ...","[5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,5,id_002852873,0.561538,-45.9,GGAAAGCGAAACGCCGAGAAGACGUAGUUCGCAGAGCGGCGUACCC...,.....(((...(((......(((((((((.((....(((.....))...,EEEEESSSIIISSSBBBBBBSSSSSSSSSBSSBBBBSSSHHHHHSS...,130,91,"[0.1706, 0.1132, 0.0, 0.0, 0.0, 0.497766666666...",0.324665,"[0, 0, 0, 0, 0, GC, CG, GC, 0, 0, 0, CG, GC, C...","[start, start, G, G, A, A, A, G, C, G, A, A, A...","[start, G, G, A, A, A, G, C, G, A, A, A, C, G,...","[G, A, A, A, G, C, G, A, A, A, C, G, C, C, G, ...","[A, A, A, G, C, G, A, A, A, C, G, C, C, G, A, ...","[5, 4, 3, 2, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 1, ..."
6,6,id_0031191b7,0.515385,-43.1,GGAAAUGUCUACAUAGGAGUGCUGCGGGACGGUAACGUCAUGACCG...,........(((((((((((.((..(((((((....))))....)))...,EEEEEEEESSSSSSSSSSSISSIISSSSSSSHHHHSSSSBBBBSSS...,130,91,"[0.1106, 0.13275, 0.0, 0.0, 0.0, 0.8138, 0.552...",0.368256,"[0, 0, 0, 0, 0, 0, 0, 0, CG, UA, AU, CG, AU, U...","[start, start, G, G, A, A, A, U, G, U, C, U, A...","[start, G, G, A, A, A, U, G, U, C, U, A, C, A,...","[G, A, A, A, U, G, U, C, U, A, C, A, U, A, G, ...","[A, A, A, U, G, U, C, U, A, C, A, U, A, G, G, ...","[8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, ..."
7,7,id_003ab2445,0.469231,-52.5,GGAAAGACUCAGAGGUGAAGGUCAUCACGGCUGAUAGGAGACUAUC...,.....(((((..........((((((.((.(((((((....)))))...,EEEEESSSSSBBBBBBBBBBSSSSSSISSISSSSSSSHHHHSSSSS...,130,91,"[0.19936666666666666, 0.19010000000000002, 0.1...",0.491034,"[0, 0, 0, 0, 0, GC, AU, CG, UA, CG, 0, 0, 0, 0...","[start, start, G, G, A, A, A, G, A, C, U, C, A...","[start, G, G, A, A, A, G, A, C, U, C, A, G, A,...","[G, A, A, A, G, A, C, U, C, A, G, A, G, G, U, ...","[A, A, A, G, A, C, U, C, A, G, A, G, G, U, G, ...","[5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,0.469231,-38.7,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[0.23125, 0.2132, 0.0, 0.35715, 0.34515, 0.212...",0.370164,"[0, 0, 0, 0, 0, 0, 0, 0, GC, CG, CG, AU, UG, G...","[start, start, G, G, A, A, A, C, U, A, G, C, C...","[start, G, G, A, A, A, C, U, A, G, C, C, A, U,...","[G, A, A, A, C, U, A, G, C, C, A, U, G, G, G, ...","[A, A, A, C, U, A, G, C, C, A, U, G, G, G, C, ...","[8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3630,3630,id_ff9bf3581,0.400000,-32.2,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[0.0, 0.3323, 0.1923, 0.0, 0.3889, 0.3611, 0.0...",0.311562,"[0, 0, 0, 0, 0, 0, 0, GC, CG, GC, CG, CG, 0, 0...","[st

In [81]:
df_train_copy3.columns

Index(['index', 'id', 'GC content', 'MFE value', 'sequence', 'structure',
       'predicted_loop_type', 'seq_length', 'seq_scored', 'rnaplfold',
       'Average pair prob', 'nucleotide pairing', 'nucleotide_Minus2',
       'nucleotide_Minus1', 'nucleotide_Plus1', 'nucleotide_Plus2',
       'distance from structured RNA'],
      dtype='object')

In [77]:
"""
explode is a function that can decompose several lists that are passed as values in a df, 
so that the elements of the lists at a given index will generate a new row in the df.
"""


def explode(list2explode, df, seq_score):
    output_df = pd.DataFrame()
    list_of_index = []
    idx_cols = df.columns.difference(list_column)
    for col in idx_cols:
        output_df[col] = np.repeat(df[col].values, seq_score)
    for column in list2explode:
        list_of_feature = []
        print(column)
        for feature in df[column]:
#             print(feature)
            for index in range(seq_score):
#                 print(index)
                list_of_feature.append(feature[index])
                
        output_df[column] = list_of_feature
    for column in list2explode:
        list_of_index = []
        for feature in df[column]:
            for index in range(seq_score):               
                list_of_index.append(index+1)
        output_df['loc in sequence'] = list_of_index
    return output_df
    
list_column_test = ['sequence', 'structure', 'predicted_loop_type', 'nucleotide pairing','nucleotide_Minus2', 
                    'nucleotide_Minus1', 'nucleotide_Plus1','nucleotide_Plus2', 'distance from structured RNA', 'rnaplfold']
    
list_column = ['sequence', 'structure', 'predicted_loop_type', 
               'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 
               'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C', 
               'nucleotide pairing','nucleotide_Minus2', 'nucleotide_Minus1', 'nucleotide_Plus1','nucleotide_Plus2', 'distance from structured RNA', 'rnaplfold']



In [83]:
# df_cop2 = pd.read_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_GC_neighboors_pairing_beforeExplode.csv')
# df_cop = pd.read_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_GC_neighboors_pairing_closest_beforeExplode.csv')

df_TrainExplode_107 = explode(list_column_test, df_107, 107)
df_TrainExplode_130 = explode(list_column_test, df_130, 130)
df_TrainExplode = pd.concat([df_TrainExplode_107, df_TrainExplode_130])
# df_TrainExplode = explode(list_column, df_train_copy3, 68)
# 
# print(output_df.columns)lens
df_TrainExplode


sequence
structure
predicted_loop_type
nucleotide pairing
nucleotide_Minus2
nucleotide_Minus1
nucleotide_Plus1
nucleotide_Plus2
distance from structured RNA
rnaplfold
sequence
structure
predicted_loop_type
nucleotide pairing
nucleotide_Minus2
nucleotide_Minus1
nucleotide_Plus1
nucleotide_Plus2
distance from structured RNA
rnaplfold


,Average pair prob,GC content,MFE value,id,index,seq_length,seq_scored,sequence,structure,predicted_loop_type,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2,distance from structured RNA,rnaplfold,loc in sequence
0,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,G,.,E,0,start,start,G,A,6,0.0,1
1,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,G,.,E,0,start,G,A,A,5,0.0,2
2,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,G,G,A,A,4,0.0,3
3,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,G,A,A,A,3,0.0,4
4,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,A,A,A,G,2,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390645,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,C,A,C,A,17,0.0,126
390646,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,C,.,E,0,A,A,A,A,18,0.0,127
390647,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,A,C,A,C,19,0.0,128
390648,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,C,A,C,end,20,0.0,129


In [84]:
df_TrainExplode.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\test_RNA_df_AfterExplode1.csv')

creating a multindex:

In [197]:
df_TrainExplode.info()
# pd.read_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_df_AfterExplode.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163200 entries, 0 to 163199
Data columns (total 29 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   GC content                    163200 non-null  float64
 1   MFE value                     163200 non-null  float64
 2   SN_filter                     163200 non-null  int64  
 3   id                            163200 non-null  object 
 4   index                         163200 non-null  int64  
 5   seq_length                    163200 non-null  int64  
 6   seq_scored                    163200 non-null  int64  
 7   signal_to_noise               163200 non-null  float64
 8   sequence                      163200 non-null  object 
 9   structure                     163200 non-null  object 
 10  predicted_loop_type           163200 non-null  object 
 11  reactivity_error              163200 non-null  float64
 12  deg_error_Mg_pH10             163200 non-nul

__b. structure dot and parenthesis to numbers__

Secondary structure: paired (0), not paired (1)




In [85]:
def replace_on_table(dataframe, to_be_replaced, replacement_values):
    dict = {}
    for index in range(len(to_be_replaced)):
        dict.update({to_be_replaced[index]: replacement_values[index]})
#     print(dict)
    dataframe["structure_number"] = dataframe["structure"].replace(to_replace=dict)
    return dataframe
        
df_TrainExplode = replace_on_table(dataframe=df_TrainExplode, to_be_replaced=["(", ")", "."], replacement_values=[0, 0, 1])
df_TrainExplode

,Average pair prob,GC content,MFE value,id,index,seq_length,seq_scored,sequence,structure,predicted_loop_type,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2,distance from structured RNA,rnaplfold,loc in sequence,structure_number
0,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,G,.,E,0,start,start,G,A,6,0.0,1,1
1,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,G,.,E,0,start,G,A,A,5,0.0,2,1
2,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,G,G,A,A,4,0.0,3,1
3,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,G,A,A,A,3,0.0,4,1
4,0.327010,0.392523,-21.7,id_00073f8be,0,107,68,A,.,E,0,A,A,A,G,2,0.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390645,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,C,A,C,A,17,0.0,126,1
390646,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,C,.,E,0,A,A,A,A,18,0.0,127,1
390647,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,A,C,A,C,19,0.0,128,1
390648,0.337158,0.415385,-34.5,id_ffda94f24,3633,130,91,A,.,E,0,C,A,C,end,20,0.0,129,1


In [ ]:
df_TrainExplode.groupby('nucleotide pairing').count()

next asignements:

- run on the whole dataframe
- clean the data
- do the maths (statistics)
- run the algorithm

In [90]:
df_TrainExplode_copy1 = df_TrainExplode.drop(columns = ['seq_length', 'seq_scored', 'index', 'structure'])
df_TrainExplode_copy1

,Average pair prob,GC content,MFE value,id,sequence,predicted_loop_type,nucleotide pairing,nucleotide_Minus2,nucleotide_Minus1,nucleotide_Plus1,nucleotide_Plus2,distance from structured RNA,rnaplfold,loc in sequence,structure_number
0,0.327010,0.392523,-21.7,id_00073f8be,G,E,0,start,start,G,A,6,0.0,1,1
1,0.327010,0.392523,-21.7,id_00073f8be,G,E,0,start,G,A,A,5,0.0,2,1
2,0.327010,0.392523,-21.7,id_00073f8be,A,E,0,G,G,A,A,4,0.0,3,1
3,0.327010,0.392523,-21.7,id_00073f8be,A,E,0,G,A,A,A,3,0.0,4,1
4,0.327010,0.392523,-21.7,id_00073f8be,A,E,0,A,A,A,G,2,0.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390645,0.337158,0.415385,-34.5,id_ffda94f24,A,E,0,C,A,C,A,17,0.0,126,1
390646,0.337158,0.415385,-34.5,id_ffda94f24,C,E,0,A,A,A,A,18,0.0,127,1
390647,0.337158,0.415385,-34.5,id_ffda94f24,A,E,0,A,C,A,C,19,0.0,128,1
390648,0.337158,0.415385,-34.5,id_ffda94f24,A,E,0,C,A,C,end,20,0.0,129,1


In [91]:
seq_dum = pd.get_dummies(df_TrainExplode_copy1['sequence'], prefix='nucleotide')
loop_dum = pd.get_dummies(df_TrainExplode_copy1['predicted_loop_type'], prefix='LoopType')
nucleotide_Minus2_dum = pd.get_dummies(df_TrainExplode_copy1['nucleotide_Minus2'], prefix = 'nucleotide_Minus2')
nucleotide_Minus1_dum = pd.get_dummies(df_TrainExplode_copy1['nucleotide_Minus1'], prefix = 'nucleotide_Minus1')
nucleotide_Plus2_dum = pd.get_dummies(df_TrainExplode_copy1['nucleotide_Plus2'], prefix = 'nucleotide_Plus2')
nucleotide_Plus1_dum = pd.get_dummies(df_TrainExplode_copy1['nucleotide_Plus1'], prefix = 'nucleotide_Plus1')
pairing_dum = pd.get_dummies(df_TrainExplode_copy1['nucleotide pairing'], prefix = 'paired')

df_TrainExplode_copy1 = pd.concat([df_TrainExplode_copy1, seq_dum, loop_dum, pairing_dum, 
                                   nucleotide_Minus2_dum, nucleotide_Minus1_dum, nucleotide_Plus2_dum, nucleotide_Plus1_dum], axis=1)
df_TrainExplode_copy2 = df_TrainExplode_copy1.drop(columns = ['sequence','predicted_loop_type', 'nucleotide pairing', 'nucleotide_Minus2', 'nucleotide_Minus1', 'nucleotide_Plus1','nucleotide_Plus2' ]) 


In [88]:
df_TrainExplode_copy2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457953 entries, 0 to 390649
Data columns (total 45 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Average pair prob             457953 non-null  float64
 1   GC content                    457953 non-null  float64
 2   MFE value                     457953 non-null  float64
 3   distance from structured RNA  457953 non-null  int64  
 4   rnaplfold                     457953 non-null  float64
 5   loc in sequence               457953 non-null  int64  
 6   structure_number              457953 non-null  int64  
 7   nucleotide_A                  457953 non-null  uint8  
 8   nucleotide_C                  457953 non-null  uint8  
 9   nucleotide_G                  457953 non-null  uint8  
 10  nucleotide_U                  457953 non-null  uint8  
 11  LoopType_B                    457953 non-null  uint8  
 12  LoopType_E                    457953 non-nul

In [92]:
df_TrainExplode_copy2.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\test_RNA_DF_AfterExplode_dumies2.csv')


In [39]:
df_TrainExplode_copy3 =df_TrainExplode_copy1.copy() 
columns = ['sequence','predicted_loop_type', 'nucleotide pairing', 'nucleotide_Minus2', 'nucleotide_Minus1', 'nucleotide_Plus1','nucleotide_Plus2' ]
for col in columns:
    df_TrainExplode_copy3[col] = df_TrainExplode_copy3[col].astype('category').values.codes

df_TrainExplode_copy3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163200 entries, 0 to 163199
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Average pair prob             163200 non-null  float64
 1   GC content                    163200 non-null  float64
 2   MFE value                     163200 non-null  float64
 3   SN_filter                     163200 non-null  int64  
 4   signal_to_noise               163200 non-null  float64
 5   sequence                      163200 non-null  int8   
 6   predicted_loop_type           163200 non-null  int8   
 7   reactivity_error              163200 non-null  float64
 8   deg_error_Mg_pH10             163200 non-null  float64
 9   deg_error_pH10                163200 non-null  float64
 10  deg_error_Mg_50C              163200 non-null  float64
 11  deg_error_50C                 163200 non-null  float64
 12  reactivity                    163200 non-nul

In [40]:
df_TrainExplode_copy3.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_DF_AfterExplode_cat2.csv')


In [ ]:
sns.pairplot(df_continuous)

In [ ]:
sns.lmplot('reactivity', 'deg_Mg_pH10', data=df_TrainExplode, hue='structure')

In [ ]:
sns.lmplot('MFE value', 'deg_Mg_pH10', data=df_TrainExplode, hue='structure')


In [ ]:
sns.lmplot('MFE value','GC content', data=df_TrainExplode)

In [ ]:
sns.violinplot('structure', 'deg_Mg_pH10', data=df_TrainExplode)


In [ ]:
sns.swarmplot('structure', 'deg_Mg_pH10', data=df_TrainExplode, hue='sequence')

In [ ]:
def find_neighboor(dataframe, number_of_neighboor):
    serie_4_neighboor = []
    serie_2_neighboor = []
    sequence = list(dataframe["sequence"])
    seq_serie = dataframe[column_name]
    if number_of_neighboor == 2:
        for index in range (len(dataframe)):
            neighboor = []
            try:
                neighboor.append(seq_serie[index-2])
            except: 
                neighboor.append("none")
            try:
                neighboor.append(seq_serie[index-1])
            except: 
                neighboor.append("none") 
            try:
                neighboor.append(seq_serie[index+1])
            except: 
                neighboor.append("none") 
            try:
                neighboor.append(seq_serie[index+2])
            except: 
                neighboor.append("none")
            finally:
                serie_4_neighboor.append(neighboor)
#         print(serie_4_neighboor)
        dataframe["serie_4_neighboor"] = serie_4_neighboor
        return dataframe
#     if number_of_neighboor == 1:
#         for index in range (len(dataframe)):
#             neighboor = []
#             try:
#                     neighboor.append(seq_serie[index-1])
#             except: 
#                 neighboor.append("none") 
#             try:
#                 neighboor.append(seq_serie[index+1])
#             except: 
#                 neighboor.append("none") 
#             finally:
#                 serie_2_neighboor.append(neighboor)
# #         print(serie_2_neighboor)
#         dataframe["serie_2_neighboor"] = serie_2_neighboor
    
    if number_of_neighboor == 1:
        neighboor5p = []
        neighboor3p = []
        for index in range (len(dataframe)):
            
            try:
                neighboor5p.append(seq_serie[index-1])
            except: 
                neighboor5p.append("start") 
            try:
                neighboor3p.append(seq_serie[index+1])
            except: 
                neighboor3p.append("end") 
#             finally:
#                 serie_2_neighboor.append(neighboor)
#         print(serie_2_neighboor)
        print(len(neighboor5p))
        dataframe["neighboor 5 prime"] = neighboor5p
        dataframe["neighboor 3 prime"] = neighboor3p
        return dataframe

    
df_TrainExplode = find_neighboor(dataframe=df_TrainExplode, number_of_neighboor=1, column_name="sequence")
df_TrainExplode.to_csv(r'C:\Users\majac\Desktop\Kaggle COVID-19 mRNA\intermediate csv files\RNA_mfe_GC_AfterExplode_neighboors.csv')
